In [7]:
import pandas as pd
import numpy as np
df = pd.read_csv("test.csv")

In [2]:
df.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
0,50883,C1,156,Owned,Individual,30,30,No,NaN,6.0,3.0,5,11934.0
1,50884,C4,7,Owned,Joint,69,68,Yes,X1,3.0,3.0,18,32204.8
2,50885,C1,564,Rented,Individual,28,28,No,X3,2.0,4.0,17,9240.0
3,50886,C3,1177,Rented,Individual,23,23,No,X3,3.0,3.0,18,9086.0
4,50887,C1,951,Owned,Individual,75,75,No,X3,NaN,NaN,5,22534.0


In [4]:
df.isnull().sum()

ID                            0
City_Code                     0
Region_Code                   0
Accomodation_Type             0
Reco_Insurance_Type           0
Upper_Age                     0
Lower_Age                     0
Is_Spouse                     0
Health Indicator           5027
Holding_Policy_Duration    8603
Holding_Policy_Type        8603
Reco_Policy_Cat               0
Reco_Policy_Premium           0
dtype: int64

In [8]:
df['City_Code'] = [int(x[1:].strip()) for x in df['City_Code']]

df['Reco_Policy_Premium']=np.log(df['Reco_Policy_Premium'])

df['Holding_Policy_Duration'][df['Holding_Policy_Duration']=='14+']=float(15)

df.head()

<ipython-input-8-79060816408c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Holding_Policy_Duration'][df['Holding_Policy_Duration']=='14+']=float(15)


,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
0,50883,1,156,Owned,Individual,30,30,No,NaN,6.0,3.0,5,9.387147
1,50884,4,7,Owned,Joint,69,68,Yes,X1,3.0,3.0,18,10.379871
2,50885,1,564,Rented,Individual,28,28,No,X3,2.0,4.0,17,9.131297
3,50886,3,1177,Rented,Individual,23,23,No,X3,3.0,3.0,18,9.114490
4,50887,1,951,Owned,Individual,75,75,No,X3,NaN,NaN,5,10.022781


In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df['Accomodation_Type'])
df['Accomodation_Type']= pd.Series(le.transform(df['Accomodation_Type']))

le = LabelEncoder()
le.fit(df['Reco_Insurance_Type'])
df['Reco_Insurance_Type']= pd.Series(le.transform(df['Reco_Insurance_Type']))

le = LabelEncoder()
le.fit(df['Is_Spouse'])
df['Is_Spouse']= pd.Series(le.transform(df['Is_Spouse']))


In [12]:
df.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
0,50883,1,156,0,0,30,0,NaN,6.0,3.0,5,9.387147
1,50884,4,7,0,1,68,1,X1,3.0,3.0,18,10.379871
2,50885,1,564,1,0,28,0,X3,2.0,4.0,17,9.131297
3,50886,3,1177,1,0,23,0,X3,3.0,3.0,18,9.114490
4,50887,1,951,0,0,75,0,X3,NaN,NaN,5,10.022781


In [11]:
df.drop(['Upper_Age'],axis=1,inplace=True)

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
misscols = ['Health Indicator','Holding_Policy_Duration']
miss = df.drop(misscols,axis=1)
col = 'Holding_Policy_Type'
miss = df.drop(misscols,axis=1)
miss.isnull().sum()

ID                        0
City_Code                 0
Region_Code               0
Accomodation_Type         0
Reco_Insurance_Type       0
Lower_Age                 0
Is_Spouse                 0
Holding_Policy_Type    8603
Reco_Policy_Cat           0
Reco_Policy_Premium       0
dtype: int64

In [14]:
tr = miss[miss[col].notnull()]
ts = miss[miss[col].isnull()]
tr.shape,ts.shape

((13202, 10), (8603, 10))

<ipython-input-15-1134cf8bd315>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr[col]=tr[col].astype('float64').astype('int64')


In [16]:
tr[col]=tr[col].astype('float64').astype('int64')
tr[col].isnull().sum()

0

In [17]:
X = tr.drop([col,'ID'],axis=1)
y = tr[col]
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,max_depth=7, random_state=0).fit(X, y)
out = clf.predict(ts.drop([col,'ID'],axis=1))


In [18]:
ts[col]=out
con = pd.concat([tr,ts])
con.sort_values(by='ID',ascending=True)

<ipython-input-18-469da847486d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts[col]=out


,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Lower_Age,Is_Spouse,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
0,50883,1,156,0,0,30,0,3,5,9.387147
1,50884,4,7,0,1,68,1,3,18,10.379871
2,50885,1,564,1,0,28,0,4,17,9.131297
3,50886,3,1177,1,0,23,0,3,18,9.114490
4,50887,1,951,0,0,75,0,3,5,10.022781
...,...,...,...,...,...,...,...,...,...,...
21800,72683,3,1044,0,0,45,0,1,18,9.673068
21801,72684,4,266,0,0,59,0,3,18,9.970679
21802,72685,12,2470,0,0,74,0,3,1,9.788974
21803,72686,10,1676,1,0,25,0,1,19,9.355998


In [19]:
con.isnull().sum()

ID                     0
City_Code              0
Region_Code            0
Accomodation_Type      0
Reco_Insurance_Type    0
Lower_Age              0
Is_Spouse              0
Holding_Policy_Type    0
Reco_Policy_Cat        0
Reco_Policy_Premium    0
dtype: int64

In [21]:
df[col]=con[col]
df.isnull().sum()

ID                            0
City_Code                     0
Region_Code                   0
Accomodation_Type             0
Reco_Insurance_Type           0
Lower_Age                     0
Is_Spouse                     0
Health Indicator           5027
Holding_Policy_Duration    8603
Holding_Policy_Type           0
Reco_Policy_Cat               0
Reco_Policy_Premium           0
dtype: int64

In [22]:
misscols = ['Holding_Policy_Duration']
col = 'Health Indicator'
miss = df.drop(misscols,axis=1)
miss.isnull().sum()

ID                        0
City_Code                 0
Region_Code               0
Accomodation_Type         0
Reco_Insurance_Type       0
Lower_Age                 0
Is_Spouse                 0
Health Indicator       5027
Holding_Policy_Type       0
Reco_Policy_Cat           0
Reco_Policy_Premium       0
dtype: int64

In [23]:
tr = miss[miss[col].notnull()]
ts = miss[miss[col].isnull()]
tr.shape,ts.shape

((16778, 11), (5027, 11))

In [25]:

tr[col].isnull().sum()

0

In [26]:
X = tr.drop([col,'ID'],axis=1)
y = tr[col]
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,max_depth=7, random_state=0).fit(X, y)
out = clf.predict(ts.drop([col,'ID'],axis=1))


In [27]:
ts[col]=out
con = pd.concat([tr,ts])
con.sort_values(by='ID',ascending=True)

<ipython-input-27-469da847486d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts[col]=out


,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
0,50883,1,156,0,0,30,0,X2,3,5,9.387147
1,50884,4,7,0,1,68,1,X1,3,18,10.379871
2,50885,1,564,1,0,28,0,X3,4,17,9.131297
3,50886,3,1177,1,0,23,0,X3,3,18,9.114490
4,50887,1,951,0,0,75,0,X3,3,5,10.022781
...,...,...,...,...,...,...,...,...,...,...,...
21800,72683,3,1044,0,0,45,0,X1,1,18,9.673068
21801,72684,4,266,0,0,59,0,X5,3,18,9.970679
21802,72685,12,2470,0,0,74,0,X3,3,1,9.788974
21803,72686,10,1676,1,0,25,0,X4,1,19,9.355998


In [29]:
con.isnull().sum()

ID                     0
City_Code              0
Region_Code            0
Accomodation_Type      0
Reco_Insurance_Type    0
Lower_Age              0
Is_Spouse              0
Health Indicator       0
Holding_Policy_Type    0
Reco_Policy_Cat        0
Reco_Policy_Premium    0
dtype: int64

In [30]:
df[col]=con[col]
df.isnull().sum()

ID                            0
City_Code                     0
Region_Code                   0
Accomodation_Type             0
Reco_Insurance_Type           0
Lower_Age                     0
Is_Spouse                     0
Health Indicator              0
Holding_Policy_Duration    8603
Holding_Policy_Type           0
Reco_Policy_Cat               0
Reco_Policy_Premium           0
dtype: int64

In [31]:
le = LabelEncoder()
le.fit(df['Health Indicator'])
df['Health Indicator']= pd.Series(le.transform(df['Health Indicator']))
df.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
0,50883,1,156,0,0,30,0,1,6.0,3,5,9.387147
1,50884,4,7,0,1,68,1,0,3.0,3,18,10.379871
2,50885,1,564,1,0,28,0,2,2.0,4,17,9.131297
3,50886,3,1177,1,0,23,0,2,3.0,3,18,9.114490
4,50887,1,951,0,0,75,0,2,NaN,3,5,10.022781


In [32]:
df.isnull().sum()

ID                            0
City_Code                     0
Region_Code                   0
Accomodation_Type             0
Reco_Insurance_Type           0
Lower_Age                     0
Is_Spouse                     0
Health Indicator              0
Holding_Policy_Duration    8603
Holding_Policy_Type           0
Reco_Policy_Cat               0
Reco_Policy_Premium           0
dtype: int64

In [33]:
misscols = []
col = 'Holding_Policy_Duration'
miss = df.drop(misscols,axis=1)
miss.isnull().sum()

ID                            0
City_Code                     0
Region_Code                   0
Accomodation_Type             0
Reco_Insurance_Type           0
Lower_Age                     0
Is_Spouse                     0
Health Indicator              0
Holding_Policy_Duration    8603
Holding_Policy_Type           0
Reco_Policy_Cat               0
Reco_Policy_Premium           0
dtype: int64

In [34]:
tr = miss[miss[col].notnull()]
ts = miss[miss[col].isnull()]
tr.shape,ts.shape

((13202, 12), (8603, 12))

In [35]:
tr[col]=tr[col].astype('float64').astype('int64')
tr.isnull().sum()

<ipython-input-35-15a5657f49b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr[col]=tr[col].astype('float64').astype('int64')


ID                         0
City_Code                  0
Region_Code                0
Accomodation_Type          0
Reco_Insurance_Type        0
Lower_Age                  0
Is_Spouse                  0
Health Indicator           0
Holding_Policy_Duration    0
Holding_Policy_Type        0
Reco_Policy_Cat            0
Reco_Policy_Premium        0
dtype: int64

In [36]:
X = tr.drop([col,'ID'],axis=1)
y = tr[col]
clf = GradientBoostingClassifier(n_estimators=150, learning_rate=0.1,max_depth=10, random_state=0).fit(X, y)
out = clf.predict(ts.drop([col,'ID'],axis=1))


In [37]:
ts[col]=out
con = pd.concat([tr,ts])
con.sort_values(by='ID',ascending=True)

<ipython-input-37-469da847486d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts[col]=out


,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
0,50883,1,156,0,0,30,0,1,6,3,5,9.387147
1,50884,4,7,0,1,68,1,0,3,3,18,10.379871
2,50885,1,564,1,0,28,0,2,2,4,17,9.131297
3,50886,3,1177,1,0,23,0,2,3,3,18,9.114490
4,50887,1,951,0,0,75,0,2,15,3,5,10.022781
...,...,...,...,...,...,...,...,...,...,...,...,...
21800,72683,3,1044,0,0,45,0,0,4,1,18,9.673068
21801,72684,4,266,0,0,59,0,4,6,3,18,9.970679
21802,72685,12,2470,0,0,74,0,2,15,3,1,9.788974
21803,72686,10,1676,1,0,25,0,3,3,1,19,9.355998


In [38]:
con.isnull().sum()

ID                         0
City_Code                  0
Region_Code                0
Accomodation_Type          0
Reco_Insurance_Type        0
Lower_Age                  0
Is_Spouse                  0
Health Indicator           0
Holding_Policy_Duration    0
Holding_Policy_Type        0
Reco_Policy_Cat            0
Reco_Policy_Premium        0
dtype: int64

In [39]:
df[col]=con[col]
df.isnull().sum()

ID                         0
City_Code                  0
Region_Code                0
Accomodation_Type          0
Reco_Insurance_Type        0
Lower_Age                  0
Is_Spouse                  0
Health Indicator           0
Holding_Policy_Duration    0
Holding_Policy_Type        0
Reco_Policy_Cat            0
Reco_Policy_Premium        0
dtype: int64

In [42]:
df.to_csv("imputedtest.csv",index=False)